In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
import sys

In [2]:
train_datagen = ImageDataGenerator(rescale=1./255,zoom_range=0.2)
val_datagen = ImageDataGenerator(rescale=1./255,zoom_range=0.2)

In [3]:
train_path=r"C:\Users\adith\Desktop\COVID19_Dataset\covid19-radiography-database\COVID-19_Radiography_Dataset\train"
test_path=r"C:\Users\adith\Desktop\COVID19_Dataset\covid19-radiography-database\COVID-19_Radiography_Dataset\test"

In [4]:
train_data= train_datagen.flow_from_directory(train_path,
                                              target_size=(299,299),
                                              batch_size=32,
                                              class_mode='categorical')
test_data = val_datagen.flow_from_directory(test_path,
                                            target_size=(299,299),
                                            batch_size=32,
                                            class_mode='categorical')

Found 42330 images belonging to 4 classes.
Found 42330 images belonging to 4 classes.


In [5]:
xception_model = Xception(input_shape=(299,299,3),include_top=False)

In [6]:
for layers in xception_model.layers:
  layers.trainable = False

In [7]:
x=Flatten()(xception_model.output)
x=Dense(256,activation='relu')(x)
output=Dense(4,activation='softmax')(x)
xception_model=Model(xception_model.input,output)

In [8]:
xception_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 299, 299, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1_conv1 (Conv2D)         │ (None, 149, 149, 32)      │             864 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1_conv1_bn               │ (None, 149, 149, 32)      │             128 │ block1_conv1[0][0]         │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1_conv1_act (Activation) │ (None, 149, 149, 32)      │               0 │ block1_conv1_bn[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1_conv2 (Conv2D)         │ (None, 147, 147, 64)      │          18,432 │ block1_conv1_act[0][0]     │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1_conv2_bn               │ (None, 147, 147, 64)      │             256 │ block1_conv2[0][0]         │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1_conv2_act (Activation) │ (None, 147, 147, 64)      │               0 │ block1_conv2_bn[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block2_sepconv1               │ (None, 147, 147, 128)     │           8,768 │ block1_conv2_act[0][0]     │
│ (SeparableConv2D)             │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block2_sepconv1_bn            │ (None, 147, 147, 128)     │             512 │ block2_sepconv1[0][0]      │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block2_sepconv2_act           │ (None, 147, 147, 128)     │               0 │ block2_sepconv1_bn[0][0]   │
│ (Activation)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block2_sepconv2               │ (None, 147, 147, 128)     │          17,536 │ block2_sepconv2_act[0][0]  │
│ (SeparableConv2D)             │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block2_sepconv2_bn            │ (None, 147, 147, 128)     │             512 │ block2_sepconv2[0][0]      │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d (Conv2D)               │ (None, 74, 74, 128)       │           8,192 │ block1_conv2_act[0][0]     │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block2_pool (MaxPooling2D)    │ (None, 74, 74, 128)       │               

 Total params: 73,291,564 (279.59 MB)

 Trainable params: 52,430,084 (200.00 MB)

 Non-trainable params: 20,861,480 (79.58 MB)

In [9]:
xception_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
my_callbacks = [EarlyStopping(patience=6)]

In [12]:
xception_model.fit(train_data,epochs=10,validation_data=test_data , steps_per_epoch=len(train_data)//20,
             validation_steps=len(test_data)//20, callbacks=my_callbacks)

Epoch 1/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 357s 5s/step - accuracy: 0.7751 - loss: 0.5862 - val_accuracy: 0.7647 - val_loss: 0.6066
Epoch 2/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 354s 5s/step - accuracy: 0.7771 - loss: 0.6041 - val_accuracy: 0.7997 - val_loss: 0.5349
Epoch 3/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 357s 5s/step - accuracy: 0.7822 - loss: 0.5791 - val_accuracy: 0.7803 - val_loss: 0.5916
Epoch 4/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 377s 6s/step - accuracy: 0.7765 - loss: 0.5839 - val_accuracy: 0.7898 - val_loss: 0.5692
Epoch 5/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 374s 6s/step - accuracy: 0.7562 - loss: 0.6068 - val_accuracy: 0.7865 - val_loss: 0.5665
Epoch 6/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 362s 6s/step - accuracy: 0.7914 - loss: 0.5744 - val_accuracy: 0.7614 - val_loss: 0.6054
Epoch 7/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 353s 5s/step - accuracy: 0.7487 - loss: 0.6554 - val_accuracy: 0.7547 - val_loss: 0.6093
Epoch 8/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 353s 5s/step - accuracy: 0.7689 - loss: 0.5869 - val_accuracy: 0.7652 - v

In [13]:
xception_model.save("xception.h5")